In [12]:
import pandas as pd 
import numpy as np
import spacy 
from py2neo import Graph, Node, Relationship 
import re, os 

In [9]:
path = os.path.join('..', '..', 'cleaned_dataset.csv')
df = pd.read_csv(path)
df.head()

,Disease,Description,Symptoms,Duration,clean_symptoms
0,Chickenpox,Chickenpox is an infection caused by the varic...,"Fever, Loss of appetite, Headache, Tiredness, ...",4-10,"fever, loss appetite, headache, tiredness, fee..."
1,Common cold,The common cold is a viral infection of your n...,"Sneezing, Stuffy nose, Runny nose, Sore throat...",3-7,"sneeze, stuffy nose, runny nose, sore throat, ..."
2,"Conjunctivitis (""pink eye“)",Pink eye (conjunctivitis) is an inflammation o...,"Pink or red color in the eye, Swelling of eyel...",7-28,"swell eyelid, itch, irritation, burn, discharg..."
3,Diarrhea,"Diarrhea — loose, watery and possibly more-fre...","Abdominal cramps or pain, Bloating, Nausea, Vo...",2-28,"abdominal cramp pain, bloat, nausea, vomit, fe..."
4,Diphtheria,Diphtheria is a serious bacterial infection th...,A thick gray membrane covering the throat and ...,7-21,"sore throat hoarseness, swollen gland neck, di..."


In [10]:
uri = 'neo4j+s://a127cef1.databases.neo4j.io'
username = 'neo4j'
password = '5ffckSQbG9tESXfS04yUO7NGQcw3SRv7wDko4wcpepc'

graph = Graph(uri, auth = (username, password))
graph 

Graph('neo4j+s://a127cef1.databases.neo4j.io:7687')

In [36]:
def clean_simp(symptoms):
    s = [i.strip().lower() for i in re.split(r",|, | , ", symptoms)] if str(symptoms) != 'nan' else []
    return s

def create_node(label, **properties):
    return Node(label, **properties)

def create_rel(a, b, rel_type):
    return Relationship(a, rel_type, b)

def create_kt():
    SIMP = {}
    for _, rdf in df.iterrows():
        disease  = rdf.Disease
        symptoms = clean_simp(rdf.clean_symptoms) 
        d_node = create_node('DISEASE', name = disease)
        for symptom in symptoms:
            if symptom not in SIMP.keys():
                SIMP[symptom] = create_node('SYMPTOM', name = symptom)
            s_node = SIMP[symptom]
            ab_rel = create_rel(d_node, s_node, 'SYMPTOM')
            ba_rel = create_rel(s_node, d_node, 'SYMPTOM_OF')
            graph.create(ab_rel)
            graph.create(ba_rel)
            
def delete_nodes():
    query = 'match (n) detach delete n'
    graph.query(query)

In [37]:
create_kt()

In [33]:
# delete_nodes()

In [35]:
SIMP

{}